In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
mainURL = "https://www.programiz.com/kotlin-programming"

page = requests.get(mainURL)
htmlContent = page.content
soup = BeautifulSoup(htmlContent, 'html.parser')

urls = []
pages = soup.find('div', {'class': 'right-bar'}).findAll(href=True)
for slug in pages[:-7]:
    urls.append('https://www.programiz.com' + slug.get('href'))

mainHeadings = []
mainContents = []
videoHeadings = []
videoURLs = []
subHeadingsH2 = []
subHeadingsH3 = []
imageURLs = []
facts = []
definitions = []
explanations = []
codes = []
snippets = []
outputs = []
tableData = []
listItems = []
data = {}


In [3]:
for url in urls:
    page = requests.get(url)
    htmlContent = page.content
    soup = BeautifulSoup(htmlContent, 'html.parser')

    mainHeading = soup.findAll('h1')  # Main Heading
    pageMainHeadings = []
    if mainHeading is not None:
        for head in mainHeading:
            pageMainHeadings.append(head.text)

    mainContent = soup.findAll('p', {"class": "editor-contents__short-description"})  # Main Content
    pageMainContents = []
    if mainContent is not None:
        for mainCon in mainContent:
            pageMainContents.append(mainCon.text)

    videoHeading = soup.findAll('div', {"class": "programiz-youtube-wrapper lite-youtube"})  # Video Heading
    pageVideoHeadings = []
    if videoHeading is not None:
        for vidHead in videoHeading:
            pageVideoHeadings.append(vidHead.find('h2').text)

    videoURL = soup.findAll('div', {"class": "programiz-youtube__container"})  # Video URL
    pageVideoURLs = []
    if videoURL is not None:
        for vid in videoURL:
            pageVideoURLs.append(vid.find('iframe').get('src'))

    subContent = soup.find('div', {"class": "content"})  # Sub Content
    if subContent is None:
        continue

    subHeadingH2 = subContent.findAll('h2')  # Sub Headings h2
    pageSubHeadingsH2 = []
    if subHeadingH2 is not None:
        for head in subHeadingH2:
            pageSubHeadingsH2.append(head.text)

    subHeadingH3 = subContent.findAll('h3')  # Sub Headings h3
    pageSubHeadingsH3 = []
    if subHeadingH3 is not None:
        for head in subHeadingH3:
            pageSubHeadingsH3.append(head.text)

    images = subContent.findAll('img')  # Images
    pageImageURLs = []
    if images is not None:
        for img in images:
            pageImageURLs.append(img.get('src'))

    paragraphs = subContent.findAll('p')  # Paragraphs
    pageFacts = []
    pageDefinitions = []
    pageExplanations = []
    if paragraphs is not None:
        for para in paragraphs:
            slug = para.text
            if 5 <= len(slug) <= 65:
                pageFacts.append(slug)
            elif 66 <= len(slug) <= 126:
                pageDefinitions.append(slug)
            elif 127 < len(slug):
                pageExplanations.append(slug)

    codeContent = subContent.findAll('pre')  # Codes, Snippets and Outputs
    pageCodes = []
    pageSnippets = []
    pageOutputs = []
    for slug in codeContent:
        if slug.code is not None:
            for cod in slug.code.text.split('\n'):
                if cod != '' and not cod.lstrip().rstrip().startswith('//'):
                    if 7 < len(cod) <= 30:
                        pageSnippets.append(cod)
                    elif 31 < len(cod):
                        pageCodes.append(cod)

        else:
            for out in slug.text.split('\n'):
                if out != '' and not out.lstrip().rstrip().startswith('//'):
                    pageOutputs.append(out)
                else:
                    continue

    tables = subContent.findAll('table')  # Tables
    pageTableData = []
    if tables is not None:
        for tb in tables:
            table = ""
            rowData = []
            tableRows = tb.findAll('tr')
            for row in tableRows:
                rowData.append(row.text[1:-1].replace('\n', '|||'))
            for row in rowData:
                table = table + str(row) + '&&&'
            pageTableData.append(table[:-3])

    allListUL = subContent.findAll('ul')  # List Items
    allListOL = subContent.findAll('ol')
    listUL = ""
    listOL = ""
    listItem = ""
    pageListItems = []
    if len(allListUL) != 0 or len(allListOL) != 0:
        if len(allListUL) != 0:
            for lis in allListUL:
                listUL = listUL + str(lis.text).replace('\n', '|||').replace('\t\t', '')
        if len(allListOL) != 0:
            for lis in allListOL:
                listOL = listOL + str(lis.text).replace('\n', '|||').replace('\t\t', '')

        if len(allListUL) != 0 and len(allListOL) == 0:
            listItem = listItem + listUL
        elif len(allListUL) == 0 and len(allListOL) != 0:
            listItem = listItem + listOL
        else:
            listItem = listItem + listUL + '&&&' + listOL
        pageListItems.append(listItem)

    # print(pageMainHeadings)
    # print(pageMainContents)
    # print(pageVideoHeadings)
    # print(pageVideoURLs)
    # print(pageSubHeadingsH2)
    # print(pageSubHeadingsH3)
    # print(pageImageURLs)
    # print(pageFacts)
    # print(pageDefinitions)
    # print(pageExplanations)
    # print(pageSnippets)
    # print(pageCodes)
    # print(pageOutputs)
    # print(pageTableData)
    # print(pageListItems)

    mainHeadings.append(pageMainHeadings)
    mainContents.append(pageMainContents)
    videoHeadings.append(pageVideoHeadings)
    videoURLs.append(pageVideoURLs)
    subHeadingsH2.append(pageSubHeadingsH2)
    subHeadingsH3.append(pageSubHeadingsH3)
    imageURLs.append(pageImageURLs)
    facts.append(pageFacts)
    definitions.append(pageDefinitions)
    explanations.append(pageExplanations)
    snippets.append(pageSnippets)
    codes.append(pageCodes)
    outputs.append(pageOutputs)
    tableData.append(pageTableData)
    listItems.append(pageListItems)


In [4]:
data = {'main_headings': mainHeadings, 'main_contents': mainContents, 'video_headings': videoHeadings,
        'video_urls': videoURLs, 'sub_headings_h2': subHeadingsH2, 'sub_headings_h3': subHeadingsH3,
        'image_urls': imageURLs, 'facts': facts, 'definitions': definitions, 'explanations': explanations,
        'snippets': snippets, 'codes': codes, 'outputs': outputs, 'table_data': tableData, 'list_items': listItems}

df = pd.DataFrame(data)
df.to_csv('Programiz_Kotlin.csv')

In [5]:
a=pd.read_csv("Programiz_Kotlin.csv")
a

,Unnamed: 0,main_headings,main_contents,video_headings,video_urls,sub_headings_h2,sub_headings_h3,image_urls,facts,definitions,explanations,snippets,codes,outputs,table_data,list_items
0,0,['Kotlin Hello World - You First Kotlin Program'],"['In this article, you will learn to write Hel...",[],[],"['Kotlin ""Hello, World!"" Program', 'Comparison...","['How this program works?', 'Few Important Not...",['//cdn.programiz.com/sites/tutorial2program/f...,"['Let\'s explore how ""Hello, World!"" program w...","['Before you write the program, make sure your...","['A ""Hello, World!"" is a simple program that o...","[' println(""Hello, World!"")', 'class HelloW...","['fun main(args : Array<String>) {', ' publ...","['Hello, World!', 'fun main(args : Array<Strin...",[],['// Hello World Program|||Any line starting w...
1,1,['Kotlin Variables and Basic Types'],"['In this tutorial, you will learn about varia...",[],[],"['Kotlin Basic Types', 'Kotlin Arrays', 'Kotli...","['How to declare a variable in Kotlin?', 'Diff...",['//cdn.programiz.com/sites/tutorial2program/f...,"['However, you can explicitly specify the type...","['As you know, a variable is a location in mem...","['To indicate the storage area, each variable ...","[' val range: Byte = 112', ' println(""$r...","['fun main(args : Array<String>) {', 'fun main...","['var language = ""French""', 'val score = 95', ...",[],['val (Immutable reference) - The variable dec...
2,2,"['Kotlin Expression, Statements and Blocks']","['In this article, you will learn about Kotlin...",[],[],"['Kotlin Expressions', 'Kotlin Statements', 'K...",[],[],"[""Let's take an example,"", 'Here, 90 + 25 is a...","['Expressions consist of variables, operators ...","['Here, if (a > b) a else b is an expression. ...","[' val a = 12', ' val b = 13', ' val ...",['fun main(args: Array<String>) { // main fun...,"['val score: Int', 'score = 90 + 25', 'val sco...",[],[]
3,3,['Kotlin Comments'],"['In this article, you will learn about Kotlin...",[],[],"['Traditional comment /* ... */', 'End of Line...",[],[],"['Similar like Java,\xa0there are two types of...",['The compiler ignores everything from\xa0//\x...,"['In programming, comments are portion of the ...","[' println(""Hello, World!"")']","['/* This is a multi-line comment.', ' * The p...","['fun main(args: Array<String>) {', ' printl...",[],['/* ... */|||// ....|||']
4,4,['Kotlin Basic Input/Output'],"['In this article, you will learn to display o...",[],[],"['Koltin Output', 'Kotlin Input']","['Difference Between println() and print()', '...",[],"['When you run the program, the output will be...","[""You can use println() and print() functions ...","['When you use println() function, it calls Sy...","[' println(""1. println "");', ' println(""...","['fun main(args : Array<String>) {', ' prin...","['Kotlin is interesting.', '1. println ', '2. ...",[],['print() - prints string inside the quotes.||...
5,5,['Kotlin if Expression'],"['In this article, you will learn to use if ex...",[],[],"['Traditional Usage of if...else', 'Kotlin if ...","['Example: Traditional Usage of if...else', 'E...",[],"['The syntax of if...else is:', 'When you run ...",['The curly braces are optional if the body of...,['if executes a certain section of code if the...,"[' val number = -10', ' if (number > 0) ...","[' print(""Positive number"")', ' ...","['if (testExpression) {', '}', 'else {', '}', ...",[],[]
6,6,['Kotlin when Expression'],"['In this article, you will learn about when c...",[],[],"['Kotlin when Construct', 'Few possibilities']",['Example: Simple when Expression'],[],"['When you run the program, the output will be...",['If none of the branch conditions are satisfi...,['The when construct in Kotlin can be thought ...,"[' val a = 12', ' val b = 5', ' val o...","[' println(""Enter operator either +, -, * o...","['Enter operator either +, -, * or /', '*', 'r...",[],[]
7,7,['Kotlin while and do...while Loop'],['Loop is used in programming to repeat a spec...,[],[],"['Kotlin while Loop',